In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import re

In [2]:
raw_data = pd.read_csv("data.csv")
raw_data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30
0,102001,0,United States,Country,NaN,127104.0,127448.0,127809.0,128546.0,129288.0,...,274766.0,278419.0,282276.0,286980.0,292503.0,298382.0,303772.0,308393.0,312486.0,316368.0
1,394913,1,"New York, NY",Msa,NY,223875.0,225213.0,226416.0,228785.0,230991.0,...,520343.0,524700.0,529431.0,534932.0,542083.0,549998.0,557578.0,563469.0,568010.0,571556.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,231151.0,231956.0,233189.0,235533.0,238046.0,...,748563.0,756432.0,767475.0,783139.0,802944.0,823842.0,837285.0,846395.0,851153.0,858357.0
3,394463,3,"Chicago, IL",Msa,IL,169017.0,169416.0,169932.0,170965.0,172060.0,...,258870.0,260970.0,263993.0,266728.0,270524.0,274664.0,278755.0,281808.0,284452.0,287131.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,130276.0,130380.0,130466.0,130678.0,130900.0,...,280524.0,284541.0,290061.0,296469.0,303787.0,311586.0,319478.0,326661.0,332734.0,338194.0


In [3]:
# Melting the dates and converting dates to datetime 
data = raw_data.melt(id_vars=["RegionName", "StateName", "RegionID", "SizeRank", "RegionType"], 
        var_name = "Date", 
        value_name ="Zillow Home Value Index")
data["Date"] = pd.to_datetime(data['Date'], format='%Y/%m/%d')
data.head()

,RegionName,StateName,RegionID,SizeRank,RegionType,Date,Zillow Home Value Index
0,United States,NaN,102001,0,Country,2000-01-31,127104.0
1,"New York, NY",NY,394913,1,Msa,2000-01-31,223875.0
2,"Los Angeles-Long Beach-Anaheim, CA",CA,753899,2,Msa,2000-01-31,231151.0
3,"Chicago, IL",IL,394463,3,Msa,2000-01-31,169017.0
4,"Dallas-Fort Worth, TX",TX,394514,4,Msa,2000-01-31,130276.0


In [4]:
# Dropping unnecessary columns 
data.drop(["RegionID", "SizeRank", "RegionType"], axis = 1, inplace = True)
data["StateName"] = data["StateName"].fillna("US")

In [5]:
# Need to impute for missing data
data["Zillow Home Value Index"].isna().sum()

48727

In [6]:
# in order to impute the n/a values, calculate the avg index for that state on that date
avg_state_date = data.groupby(["StateName", "Date"]).mean()
avg_state_date.reset_index(level = ["StateName", "Date"], inplace = True)
avg_state_date.rename(columns ={"Zillow Home Value Index": "Average Zillow Home Value Index for State on Date"}, inplace = True)
avg_state_date["Average Zillow Home Value Index for State on Date"].isna().sum()

342

In [7]:
# Merging back to original dataframe and filling the holes
data = pd.merge(data, avg_state_date, how = "left", on = ["StateName", "Date"])
data["Zillow Home Value Index"].fillna(data["Average Zillow Home Value Index for State on Date"], inplace = True)
data["Zillow Home Value Index"].isna().sum()

3468

In [8]:
# Since some states may have no data for that date, just get the average for the region across Time
avg_region = data.groupby(["RegionName"]).mean()
avg_region.drop(["Average Zillow Home Value Index for State on Date"], axis =1, inplace = True)
avg_region.reset_index(level = ["RegionName"], inplace = True)
avg_region.rename(columns ={"Zillow Home Value Index": "Average Zillow Home Value Index for Region across Time"}, inplace = True)
data = pd.merge(data, avg_region, how = "left", on = ["RegionName"])
data["Zillow Home Value Index"].fillna(data["Average Zillow Home Value Index for Region across Time"], inplace = True)
data["Zillow Home Value Index"].isna().sum()

0

In [9]:
data.head()

,RegionName,StateName,Date,Zillow Home Value Index,Average Zillow Home Value Index for State on Date,Average Zillow Home Value Index for Region across Time
0,United States,US,2000-01-31,127104.0,127104.000000,194320.676806
1,"New York, NY",NY,2000-01-31,223875.0,79925.461538,407514.752852
2,"Los Angeles-Long Beach-Anaheim, CA",CA,2000-01-31,231151.0,221478.900000,505735.870722
3,"Chicago, IL",IL,2000-01-31,169017.0,90254.285714,222440.041825
4,"Dallas-Fort Worth, TX",TX,2000-01-31,130276.0,103721.555556,177825.041825
